In [1]:
import math
import pandas as pd
from lenskit import topn
from sklearn.metrics import ndcg_score
from statistics import mean, variance
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.metrics.topn import ndcg
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
preprocessed_dataset_folder = "../preprocessed_dataset"

In [3]:
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv") 
display(ratings_df.head(15))


,user,item,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,70,3.0,964982400
4,1,101,5.0,964980868
5,1,110,4.0,964982176
6,1,151,5.0,964984041
7,1,157,5.0,964984100
8,1,163,5.0,964983650
9,1,216,5.0,964981208


In [4]:
movies_df = pd.read_csv(preprocessed_dataset_folder+"/movies.csv", index_col="item")
display(movies_df.head(14))

,title,year,genres,plot
item,,,,
1,toy story,1995,Adventure|Animation|Children|Comedy|Fantasy,In a world where toys are living things who pr...
2,jumanji,1995,Adventure|Children|Fantasy,"In 1869, near Brantford, New Hampshire, two br..."
3,grumpier old men,1995,Comedy|Romance,The feud between Max (Walter Matthau) and John...
4,waiting to exhale,1995,Comedy|Drama|Romance,"""Friends are the People who let you be yoursel..."
5,father of the bride part ii,1995,Comedy,The film begins five years after the events of...
6,heat,1995,Action|Crime|Thriller,"Neil McCauley, a career criminal, hires Waingr..."
7,sabrina,1995,Comedy|Romance,Sabrina Fairchild is the young daughter of the...
8,tom and huck,1995,Adventure|Children,The movie opens with Injun Joe (Eric Schweig) ...
9,sudden death,1995,Action,Darren McCord (Jean-Claude Van Damme) is a Fre...


In [5]:
# selection of random user and create groups of size 4
users_ratings = ratings_df.groupby(['user']).count()

groupsize = 4

selected = users_ratings['rating'] > 200
selected_users = users_ratings.loc[selected]
random_selected = selected_users.sample(groupsize) # sample() returns a random row from the dataframe. The returned object is a dataframe with one row. If you pass a number as argument you specify to select more than one row.
select_column_df = random_selected.reset_index()['user'] # reset_index() create a new index, and the user became a column. Then, we can filter using the column name
selected_users = select_column_df.values # iloc select by index, since our dataframe only has one row we read it from the index 0
selected_user = select_column_df.iloc[0] 

print("Selected users: " + str(selected_users))

Selected users: [339 380 382 567]


In [6]:
#Evaluation of a collaborative filtering approach using a holduot strategy with 80% for the training set and
#remaining for test set
train_df, test_df = train_test_split(ratings_df, test_size=0.2)

user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(train_df)

test_df['predicted_rating'] = recsys.predict(test_df)

test_df['relevant'] = test_df['rating'].apply(lambda x: 1 if x>3 else 0)
test_df['predicted_relevant'] = test_df['predicted_rating'].apply(lambda x: 1 if x>3 else 0)

predicted_rating = test_df['predicted_rating'].apply(lambda x: 1 if x>3 else 0)

y_test = list(test_df['relevant'])
y_pred = list(test_df['predicted_relevant'])
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")

print("Precision:\t" + str(precision) + 
      "\nRecall:\t" + str(recall) + 
      "\nFscore:\t" + str(fscore))


BLAS using multiple threads - can cause oversubscription
found 1 potential runtime problems - see https://boi.st/lkpy-perf


Precision:	0.7076252723311547
Recall:	0.8547368421052631
Fscore:	0.7742550655542312


In [7]:
group_list = []
for i in selected_users:
    selected_movies = recsys.recommend(i, 10)
    group_list.append(selected_movies)

In [8]:
group_df = pd.concat(group_list)
group_df.drop_duplicates(subset=["item"], inplace=True)

In [9]:
lst = []
for i in selected_users:
    print(i)
    grp = group_df.copy()
    grp["user"] = i
    grp["pred"] = recsys.predict_for_user(i, grp["item"]).values
    lst.append(grp)

339
380
382
567


In [10]:
final = pd.concat(lst)

In [11]:
#Here I implement both least misery and additive aggregations strategies
final.sort_values(by=["pred"], inplace=True, ascending=False)
final

gbadd = final.groupby("item", as_index=False)["pred"].sum() #Additive strat
gbleast = final.groupby("item", as_index=False)["pred"].min() #Least misery strat


In [12]:
# considering 5 recommendations of add strat
final_recommendationsAdd = gbadd.sort_values(by=["pred"], ascending=False).head(5)
final_recommendationsAdd

,item,pred
10,3451,18.473165
9,3030,18.309451
2,1204,17.655362
15,3836,17.571734
7,2202,17.399665


In [13]:
# considering 5 recommendations of least strat
final_recommendationsleast = gbleast.sort_values(by=["pred"], ascending=False).head(5)
final_recommendationsleast

,item,pred
14,3606,5.180631
4,1236,4.736642
20,51931,4.732605
11,3494,4.601737
27,177593,4.495337


In [14]:
evaluateAdd = final_recommendationsAdd.copy()
for i in selected_users:
    evaluateAdd[i] = recsys.predict_for_user(i, final_recommendationsAdd["item"]).values
evaluateAdd
# if the scores is not possible,replace with 0
evaluateAdd.fillna(0, inplace=True)
recommended_ordering = pd.DataFrame(evaluateAdd[["item","pred"]])
recommended_ordering

,item,pred
10,3451,18.473165
9,3030,18.309451
2,1204,17.655362
15,3836,17.571734
7,2202,17.399665


In [15]:
evaluateleast = final_recommendationsleast.copy()
for i in selected_users:
    evaluateleast[i] = recsys.predict_for_user(i, final_recommendationsleast["item"]).values
evaluateleast
#same here but null values
evaluateleast.fillna(0, inplace=True)
recommended_orderingleast = pd.DataFrame(evaluateleast[["item","pred"]])
recommended_orderingleast

,item,pred
14,3606,5.180631
4,1236,4.736642
20,51931,4.732605
11,3494,4.601737
27,177593,4.495337


In [16]:
#decupled evaluation for additive strategy
ndcg_lstAdd = []

for i in selected_users:
    evaluateAdd.sort_values(by=[i], ascending=False, inplace=True)
    ev = pd.DataFrame(evaluateAdd[["item", i]])
    val = ndcg_score([ev.item], [recommended_ordering.item])
    ndcg_lstAdd.append(val)

ndcg_lstAdd

[0.9293685767637416,
 0.9938359879102177,
 0.9439968444469227,
 0.8213783475362092]

In [17]:
#decoupled evaluation for least misery strategy
ndcg_lstleast = []

for i in selected_users:
    evaluateleast.sort_values(by=[i], ascending=False, inplace=True)
    ev = pd.DataFrame(evaluateleast[["item", i]])
    val = ndcg_score([ev.item], [recommended_orderingleast.item])
    ndcg_lstleast.append(val)

ndcg_lstleast

[0.7833891010929883, 0.620619385206245, 0.5937611944304179, 0.5937611944304179]

In [18]:
#then I take the mean of the ndcg score for add strat
mean(ndcg_lstAdd)

0.9221449391642728

In [19]:
#and then the mean of the ndcg score for least strat
mean(ndcg_lstleast)

0.6478827187900172

After running the code several times, it seems like there is no big differences between least misery strategy and additive strategy, sometimes it shows that additive is better and sometimes it shows that least misery is better. So to make my results more accurate, I have made a method pipeline in an other file than this one, where I have made 3 times 1000 run and took the mean of it, and it appears that additive strategy is slightly better than least misery strategy but not really better, just slighlty. To check those results, please look at the pipeline Alternative Resit jupyter file.